---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    # 4 The Grammar
    # 4.1 Cardinality
    ZERO_OR_MORE = '*'
    ONE_OR_MORE = "+"
    ZERO_OR_ONE = '?'
    EXACTLY_TWO = "{2}"
    ONE_OR_TWO = "{1,2}"
    EXACTLY_ONE = '{1}'
    
    # 4.2 Groups and Classes
    GROUP = r"({})"
    NAMED = r"(?P<{}>{})"
    CLASS = "[{}]"
    NEGATIVE_LOOKAHEAD = "(?!{})"
    NEGATIVE_LOOKBEHIND = "(?<!{})"
    POSITIVE_LOOKAHEAD = "(?={})"
    POSITIVE_LOOKBEHIND = "(?<={})"
    ESCAPE = "\{}"
    
    # 4.3 Numbers
    DIGIT = r"\d"
    ONE_DIGIT  = DIGIT + EXACTLY_ONE
    ONE_OR_TWO_DIGITS = DIGIT + ONE_OR_TWO
    NON_DIGIT = NEGATIVE_LOOKAHEAD.format(DIGIT)
    TWO_DIGITS = DIGIT + EXACTLY_TWO
    THREE_DIGITS = DIGIT + "{3}"
    EXACTLY_TWO_DIGITS = DIGIT + EXACTLY_TWO + NON_DIGIT
    FOUR_DIGITS = DIGIT + r"{4}" + NON_DIGIT
    
    # 4.4 String Literals
    SLASH = r"/"
    OR = r'|'
    LOWER_CASE = "a-z"
    SPACE = "\s"
    DOT = "."
    DASH = "-"
    COMMA = ","
    PUNCTUATION = CLASS.format(DOT + COMMA + DASH)
    EMPTY_STRING = ""
    
    # 4.5 Dates
    # These are parts to build up the date-expressions.
    MONTH_SUFFIX = (CLASS.format(LOWER_CASE) + ZERO_OR_MORE
                    + CLASS.format(SPACE + DOT + COMMA + DASH) + ONE_OR_TWO)
    MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
    MONTHS = [month + MONTH_SUFFIX for month in MONTH_PREFIXES]
    MONTHS = GROUP.format(OR.join(MONTHS))
    DAY_SUFFIX = CLASS.format(DASH + COMMA + SPACE) + ONE_OR_TWO
    DAYS = ONE_OR_TWO_DIGITS + DAY_SUFFIX
    YEAR = FOUR_DIGITS
    
    # This is for dates like Mar 21st, 2009, those with suffixes on the days.
    CONTRACTED = (ONE_OR_TWO_DIGITS
                  + LOWER_CASE
                  + EXACTLY_TWO
                  )
    CONTRACTION = NAMED.format("contraction",
                               MONTHS
                               + CONTRACTED
                               + DAY_SUFFIX
                               + YEAR)
    
    # This is for dates that have no days in them, like May 2009.
    NO_DAY_BEHIND = NEGATIVE_LOOKBEHIND.format(DIGIT + SPACE)
    NO_DAY = NAMED.format("no_day", NO_DAY_BEHIND + MONTHS + YEAR)
    
    # This is for the most common form (that I use) - May 21, 2017.
    WORDS = NAMED.format("words", MONTHS + DAYS + YEAR)
    
    BACKWARDS = NAMED.format("backwards", ONE_OR_TWO_DIGITS + SPACE + MONTHS + YEAR)
    
    slashed = SLASH.join([ONE_OR_TWO_DIGITS,
                      ONE_OR_TWO_DIGITS,
                      EXACTLY_TWO_DIGITS])
    dashed = DASH.join([ONE_OR_TWO_DIGITS,
                        ONE_OR_TWO_DIGITS,
                        EXACTLY_TWO_DIGITS])
    TWENTIETH_CENTURY = NAMED.format("twentieth",
                                     OR.join([slashed, dashed]))
    NUMERIC = NAMED.format("numeric",
                       SLASH.join([ONE_OR_TWO_DIGITS,
                                   ONE_OR_TWO_DIGITS,
                                   FOUR_DIGITS]))
    NO_PRECEDING_SLASH = NEGATIVE_LOOKBEHIND.format(SLASH)
    NO_PRECEDING_SLASH_DIGIT = NEGATIVE_LOOKBEHIND.format(CLASS.format(SLASH + DIGIT))
    NO_ONE_DAY = (NO_PRECEDING_SLASH_DIGIT
                  + ONE_DIGIT
                  + SLASH
                  + FOUR_DIGITS)
    NO_TWO_DAYS = (NO_PRECEDING_SLASH
                   + TWO_DIGITS
                   + SLASH
                   + FOUR_DIGITS)
    NO_DAY_NUMERIC = NAMED.format("no_day_numeric",
                                  NO_ONE_DAY
                                  + OR
                                  + NO_TWO_DAYS
                                  )
    CENTURY = GROUP.format('19' + OR + "20") + TWO_DIGITS
    DIGIT_SLASH = DIGIT + SLASH
    DIGIT_DASH = DIGIT + DASH
    DIGIT_SPACE = DIGIT + SPACE
    LETTER_SPACE = CLASS.format(LOWER_CASE) + SPACE
    COMMA_SPACE = COMMA + SPACE
    YEAR_PREFIX = NEGATIVE_LOOKBEHIND.format(OR.join([
        DIGIT_SLASH,
        DIGIT_DASH,
        DIGIT_SPACE,
        LETTER_SPACE,
        COMMA_SPACE,
    ]))
    YEAR_ONLY = NAMED.format("year_only",
                             YEAR_PREFIX + CENTURY
    )
    IN_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format('iI') + 'n' + SPACE) + CENTURY
    SINCE_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format("Ss") + 'ince' + SPACE) + CENTURY
    AGE = POSITIVE_LOOKBEHIND.format("Age" + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
    AGE_COMMA = POSITIVE_LOOKBEHIND.format("Age" + COMMA + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
    OTHERS = ['delivery', "quit", "attempt", "nephrectomy", THREE_DIGITS]
    OTHERS = [POSITIVE_LOOKBEHIND.format(label + SPACE) + CENTURY for label in OTHERS]
    OTHERS = OR.join(OTHERS)
    LEFTOVERS_PREFIX = OR.join([IN_PREFIX, SINCE_PREFIX, AGE, AGE_COMMA]) + OR + OTHERS
    LEFTOVERS = NAMED.format("leftovers", LEFTOVERS_PREFIX)
    DATE = NAMED.format("date", OR.join([NUMERIC,
                                     TWENTIETH_CENTURY,
                                     WORDS,
                                     BACKWARDS,
                                     CONTRACTION,
                                     NO_DAY,
                                     NO_DAY_NUMERIC,
                                     YEAR_ONLY,
                                     LEFTOVERS]))
    
    def twentieth_century(date):
        """adds a 19 to the year

        Args:
         date (re.Regex): Extracted date
        """
        month, day, year = date.group(1).split(SLASH)
        year = "19{}".format(year)
        return SLASH.join([month, day, year])

    def take_two(line):
        match = re.search(TWENTIETH_CENTURY, line)
        if match:
            return twentieth_century(match)
        return line

    def extract_and_count(expression, df, name):
        """extract all matches and report the count

        Args:
         expression (str): regular expression to match
         df (pd.Series): df with dates to extratc
         name (str): name of the group for the expression

        Returns:
         tuple (pd.Series, int): extracted dates, count
        """
        extracted = df.str.extractall(expression)[name]
        count = len(extracted)
        print("'{}' matched {} rows".format(name, count))
        return extracted, count

    numeric, numeric_count = extract_and_count(NUMERIC, df, 'numeric')
    # 'numeric' matched 25 rows
    twentieth, twentieth_count = extract_and_count(TWENTIETH_CENTURY, df, 'twentieth')
    # 'twentieth' matched 100 rows
    words, words_count = extract_and_count(WORDS, df, 'words')
    # 'words' matched 34 rows
    backwards, backwards_count = extract_and_count(BACKWARDS, df, 'backwards')
    # 'backwards' matched 69 rows
    contraction_df, contraction = extract_and_count(CONTRACTION, df, 'contraction')
    # 'contraction' matched 0 rows
    no_day, no_day_count = extract_and_count(NO_DAY, df, 'no_day')
    # 'no_day' matched 115 rows
    no_day_numeric, no_day_numeric_count = extract_and_count(NO_DAY_NUMERIC, df,
                                                             "no_day_numeric")
    # 'no_day_numeric' matched 112 rows
    year_only, year_only_count = extract_and_count(YEAR_ONLY, df, "year_only")
    # 'year_only' matched 15 rows
    leftovers, leftovers_count = extract_and_count(LEFTOVERS, df, "leftovers")
    # 'leftovers' matched 30 rows
    
    found = df.str.extractall(DATE)
    total_found = len(found.date)

    print("Total Found: {}".format(total_found))
    print("Remaining: {}".format(len(df) - total_found))
    print("Discrepancy: {}".format(total_found - (numeric_count
                                                  + twentieth_count
                                                  + words_count
                                                  + backwards_count
                                                  + contraction
                                                  + no_day_count
                                                  + no_day_numeric_count
                                                  + year_only_count
                                                  + leftovers_count)))
    
    missing = [label for label in df.index if label not in found.index.levels[0]]
    try:
        print(missing[0], df.loc[missing[0]])
    except IndexError:
        print("all rows matched")
    
    def clean(source, expression, replacement, sample=5):
        """applies the replacement to the source

        as a side-effect shows sample rows before and after

        Args:
         source (pd.Series): source of the strings
         expression (str): regular expression to match what to replace
         replacement: function or expression to replace the matching expression
         sample (int): number of randomly chosen examples to show

        Returns:
         pd.Series: the source with the replacement applied to it
        """
        print("Random Sample Before:")
        print(source.sample(sample))
        cleaned = source.str.replace(expression, replacement)
        print("\nRandom Sample After:")
        print(cleaned.sample(sample))
        print("\nCount of cleaned: {}".format(len(cleaned)))
        assert len(source) == len(cleaned)
        return cleaned

    def clean_punctuation(source, sample=5):
        """removes punctuation

        Args:
         source (pd.Series): df to clean
         sample (int): size of sample to show

        Returns:
         pd.Series: source with punctuation removed
        """
        print("Cleaning Punctuation")
        if any(source.str.contains(PUNCTUATION)):
            source = clean(source, PUNCTUATION, EMPTY_STRING)
        return source
    
    LONG_TO_SHORT = dict(January="Jan",
                     February="Feb",
                     March="Mar",
                     April="Apr",
                     May="May",
                     June="Jun",
                     July="Jul",
                     August="Aug",
                     September="Sep",
                     October="Oct",
                     November="Nov",
                     December="Dec")

    # it turns out there are spelling errors in the df so this has to be fuzzy
    LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
                                        + CLASS.format(LOWER_CASE)
                                        + ZERO_OR_MORE
                                        for month in LONG_TO_SHORT.values()])

    def long_month_to_short(match):
        """convert long month to short

        Args:
         match (re.Match): object matching a long month

        Returns:
         str: shortened version of the month
        """
        return match.group(match.lastindex)

    def convert_long_months_to_short(source, sample=5):
        """convert long month names to short

        Args:
         source (pd.Series): df with months
         sample (int): size of sample to show

        Returns:
         pd.Series: df with short months
        """
        return clean(source,
                     LONG_TO_SHORT_EXPRESSION,
                     long_month_to_short)
    
    def add_month_date(match):
        """adds 01/01 to years

        Args:
         match (re.Match): object that only matched a 4-digit year

        Returns:
         str: 01/01/YYYY
        """
        return "01/01/" + match.group()

    def add_january_one(source):
        """adds /01/01/ to year-only dates

        Args:
         source (pd.Series): df with the dates

        Returns:
         pd.Series: years in source with /01/01/ added
        """
        return clean(source, YEAR_ONLY, add_month_date)

    two_digit_expression = GROUP.format(ONE_OR_TWO_DIGITS) + POSITIVE_LOOKAHEAD.format(SLASH)

    def two_digits(match):
        """add a leading zero if needed

        Args:
         match (re.Match): match with one or two digits

        Returns:
         str: the matched string with leading zero if needed
        """
        # for some reason the string-formatting raises an error if it's a string
        # so cast it to an int
        return "{:02}".format(int(match.group()))

    def clean_two_digits(source, sample=5):
        """makes sure source has two-digits

        Args:
         source (pd.Series): df with digit followed by slash
         sample (int): number of samples to show

        Returns:
         pd.Series: source with digits coerced to two digits
        """
        return clean(source, two_digit_expression, two_digits, sample)

    def clean_two_digits_isolated(source, sample=5):
        """cleans two digits that are standalone

        Args:
         source (pd.Series): source of the df
         sample (int): number of samples to show

        Returns:
         pd.Series: converted df
        """
        return clean(source, ONE_OR_TWO_DIGITS, two_digits, sample)

    digits = ("{:02}".format(month) for month in range(1, 13))
    MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
    SHORT_MONTHS_EXPRESSION = OR.join((GROUP.format(month) for month in MONTH_TO_DIGITS))
    def month_to_digits(match):
        """converts short month to digits

        Args:
         match (re.Match): object with short-month

        Returns:
         str: month as two-digit number (e.g. Jan -> 01)
        """
        return MONTH_TO_DIGITS[match.group()]

    def convert_short_month_to_digits(source, sample=5):
        """converts three-letter months to two-digits

        Args:
         source (pd.Series): df with three-letter months
         sample (int): number of samples to show

        Returns:
         pd.Series: source with short-months coverted to digits
        """
        return clean(source,
                     SHORT_MONTHS_EXPRESSION,
                     month_to_digits,
                     sample)

    def clean_months(source, sample=5):
        """clean up months (which start as words)

        Args:
         source (pd.Series): source of the months
         sample (int): number of random samples to show
        """
        cleaned = clean_punctuation(source)

        print("Converting long months to short")
        cleaned = clean(cleaned,
                        LONG_TO_SHORT_EXPRESSION,
                        long_month_to_short, sample)

        print("Converting short months to digits")
        cleaned = clean(cleaned,
                        SHORT_MONTHS_EXPRESSION,
                        month_to_digits, sample)
        return cleaned
    
    def frame_to_series(frame, index_source, samples=5):
        """re-combines df-frame into a series

        Args:
         frame (pd.DataFrame): frame with month, day, year columns
         index_source (pd.series): source to copy index from
         samples (index): number of random entries to print when done

        Returns:
         pd.Series: series with dates as month/day/year
        """
        combined = frame.month + SLASH + frame.day + SLASH + frame.year
        combined.index = index_source.index
        print(combined.sample(samples))
        return combined

    year_only_cleaned = add_january_one(year_only)
    leftovers_cleaned = add_january_one(leftovers)
    cleaned = pd.concat([year_only_cleaned, leftovers_cleaned])
    no_day_numeric_cleaned = clean_two_digits(no_day_numeric)
    
    no_day_numeric_cleaned = clean(no_day_numeric_cleaned,
                               SLASH,
                               lambda m: "/01/")

    original = len(cleaned)
    cleaned = pd.concat([cleaned, no_day_numeric_cleaned])

    no_day_cleaned = clean_months(no_day)
    
    no_day_cleaned = clean(no_day_cleaned,
                       SPACE + ONE_OR_MORE,
                       lambda match: "/01/")

    original = len(cleaned)
    cleaned = pd.concat([cleaned, no_day_cleaned])
    assert len(cleaned) == no_day_count + original
    
    frame = pd.DataFrame(backwards.str.split().tolist(),
                         columns="day month year".split())
    frame.day = clean_two_digits(frame.day)
    frame.month = clean_months(frame.month)
    backwards_cleaned = frame_to_series(frame, backwards)
    
    original = len(cleaned)
    cleaned = pd.concat([cleaned, backwards_cleaned])
    assert len(cleaned) == original + backwards_count
    
    frame = pd.DataFrame(words.str.split().tolist(), columns="month day year".split())
    frame.month = clean_months(frame.month)
    frame.day = clean_punctuation(frame.day)

    words_cleaned = frame_to_series(frame, words)
    original = len(cleaned)
    cleaned = pd.concat([cleaned, words_cleaned])
    assert len(cleaned) == original + words_count

    twentieth_cleaned = twentieth.str.replace(DASH, SLASH)
    frame = pd.DataFrame(twentieth_cleaned.str.split(SLASH).tolist(),
                             columns=["month", "day", "year"])
    frame.month = clean_two_digits_isolated(frame.month)
    frame.day = clean_two_digits_isolated(frame.day)
    frame.head()
    
    frame.year = clean(frame.year, TWO_DIGITS, lambda match: "19" + match.group())
    twentieth_cleaned = frame_to_series(frame, twentieth)
    original = len(cleaned)
    cleaned = pd.concat([cleaned, twentieth_cleaned])
    assert len(cleaned) == original + twentieth_count
    
    has_dashes = numeric.str.contains(DASH)
    frame = pd.DataFrame(numeric.str.split(SLASH).tolist(),
                             columns="month day year".split())
    frame.month = clean_two_digits_isolated(frame.month)
    frame.day = clean_two_digits_isolated(frame.day)
    numeric_cleaned = frame_to_series(frame, numeric)
    original = len(cleaned)
    cleaned = pd.concat([cleaned, numeric_cleaned])
    assert len(cleaned) == original + numeric_count
    
    cleaned = pd.concat([numeric_cleaned,
                         twentieth_cleaned,
                         words_cleaned,
                         backwards_cleaned,
                         no_day_cleaned,
                         no_day_numeric_cleaned,
                         year_only_cleaned,
                         leftovers_cleaned,
    ])
    print(len(cleaned))
    print(cleaned.head())
    assert len(cleaned) == len(df)
    
    datetimes = pd.to_datetime(cleaned, format="%m/%d/%Y")
    sorted_dates = datetimes.sort_values()
    
    return pd.Series(sorted_dates.index.labels[0])


In [3]:
date_sorter()

'numeric' matched 25 rows
'twentieth' matched 100 rows
'words' matched 34 rows
'backwards' matched 69 rows
'contraction' matched 0 rows
'no_day' matched 115 rows
'no_day_numeric' matched 112 rows
'year_only' matched 15 rows
'leftovers' matched 30 rows
Total Found: 500
Remaining: 0
Discrepancy: 0
all rows matched
Random Sample Before:
     match
472  0        2010
470  0        1983
473  0        1975
481  0        1974
478  0        1993
Name: year_only, dtype: object

Random Sample After:
     match
476  0        01/01/1989
470  0        01/01/1983
486  0        01/01/1973
497  0        01/01/2008
495  0        01/01/1979
Name: year_only, dtype: object

Count of cleaned: 15
Random Sample Before:
     match
459  0        1998
464  0        2016
492  0        1986
460  0        2012
482  0        1990
Name: leftovers, dtype: object

Random Sample After:
     match
477  0        01/01/1994
468  0        01/01/1997
499  0        01/01/1980
474  0        01/01/1972
464  0        01/01/2016

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    480
478    286
479    431
480    279
481    198
482    463
483    381
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int16